In [ ]:
#handle warnings
import warnings
warnings.filterwarnings("ignore")

import shapely                 #needed to set geopandas geometry 
from shapely.wkt import loads  #needed to set geopandas geometry

import pandas as pd            #for DataFrame and analysis
import geopandas as gpd        #for mapping and spatial aggregation
import mapclassify as mp   #to view quantitative classification summaries

import numpy as np        #for working with arrays from mapclassify

import matplotlib.pyplot as plt  #use matplotlib to modify the plots (e.g. make them larger) 
import palettable.colorbrewer.sequential as pcs  #for colorBrewer palettes

In [ ]:

df_imd = pd.read_csv("IMD2019_Index_of_Multiple_Deprivation.csv")


lsoa_gdf = gpd.read_file("LSOA_2011_London_gen_MHW.shp")


print("IMD DataFrame Columns:", df_imd.columns)
print("LSOA GeoDataFrame Columns:", lsoa_gdf.columns)


common_column_left = 'LSOA11CD'
common_column_right = 'LSOA code (2011)'


merged_gdf = lsoa_gdf.merge(df_imd, left_on=common_column_left, right_on=common_column_right, how='left')


print("Merged GeoDataFrame Columns:", merged_gdf.columns)



In [ ]:
lsoa_gdf.head()

In [ ]:
lsoa_gdf = gpd.read_file("LSOA_2011_London_gen_MHW.shp")


print("IMD DataFrame Columns:", df_imd.columns)
print("LSOA GeoDataFrame Columns:", lsoa_gdf.columns)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

merged_gdf.plot(
    column='Index of Multiple Deprivation (IMD) Decile',
    cmap='OrRd',
    linewidth=0.2,
    edgecolor='grey',
    legend=True,
    ax=ax,
    legend_kwds={
        'shrink': 0.5,           
        'label': 'IMD Decile',   
        'orientation': 'vertical'  
    },
    missing_kwds={'color': 'lightgrey', "label": "No Data"}
)

ax.set_title("London LSOAs Index of Multiple Deprivation Decile, 2019", fontsize=15)
ax.set_axis_off()
plt.show()

In [ ]:
import matplotlib.pyplot as plt 
from matplotlib import colors 
# Analysis
import geopandas as gpd 
import pandas as pd
from libpysal import weights 
from pysal.explore import esda
import numpy as np

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import shapely                 #needed to set geopandas geometry 
from shapely.wkt import loads  #needed to set geopandas geometry

import pandas as pd            #for DataFrame and analysis
import geopandas as gpd        #for mapping and spatial aggregation
import mapclassify as mp   #to view quantitative classification summaries

import numpy as np        #for working with arrays from mapclassify

import matplotlib.pyplot as plt  #use matplotlib to modify the plots (e.g. make them larger) 
import palettable.colorbrewer.sequential as pcs  #for colorBrewer palettes

In [ ]:
print(df_imd.columns.tolist())


In [ ]:
merged_gdf = lsoa_gdf.merge(df_imd, left_on="LSOA11CD", right_on="LSOA code (2011)")

In [ ]:
w = weights.KNN.from_dataframe(merged_gdf, k=8)
w.transform = 'R'

In [ ]:
merged_gdf['Index of Multiple Deprivation (IMD) Rank'] = (
    merged_gdf['Index of Multiple Deprivation (IMD) Rank']
    .str.replace(',', '', regex=False)   # Remove commas
    .astype(float)                       # Convert to float (or int if you prefer)
)

merged_gdf['Index of Multiple Deprivation (IMD) Rank'] = (
    merged_gdf['Index of Multiple Deprivation (IMD) Rank']
    .str.replace(',', '', regex=False)   # Remove commas
    .astype(float)                       # Convert to float (or int if you prefer)
)

In [ ]:
moran = esda.moran.Moran(merged_gdf['Index of Multiple Deprivation (IMD) Rank'], w)

In [ ]:
round(moran.I,3)

In [ ]:
moran.p_sim

In [ ]:
plt.hist(moran.sim, 10, facecolor='lightblue', edgecolor='black')
plt.vlines(moran.I, 0, 350, color='r', linestyle="--")
plt.vlines(moran.EI, 0, 350)
plt.xlabel("Moran's I")
plt.ylabel("Count")
plt.show()

In [ ]:
merged_gdf['IMD_Rank_lag'] = weights.spatial_lag.lag_spatial(w, merged_gdf['Index of Multiple Deprivation (IMD) Rank'])

In [ ]:
def standardize(df, var):
    newname = var + '_z'
    df[newname] = (df[var] - df[var].mean()) / df[var].std()

standardize(merged_gdf,'Index of Multiple Deprivation (IMD) Rank')
standardize(merged_gdf, 'IMD_Rank_lag')

In [ ]:
merged_gdf.tail()

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(8,10))

ax=ax.flatten()


for i, p in enumerate(['Index of Multiple Deprivation (IMD) Rank',
                       'Index of Multiple Deprivation (IMD) Rank_z','IMD_Rank_lag','IMD_Rank_lag_z']):
    
    merged_gdf.plot(column=p, cmap='viridis', 
                   scheme='quantiles', k=5,
                   linewidth=0.,
                   legend=True, legend_kwds={"title":p, "loc": 3}, ax=ax[i]
                  )
    ax[i].set_axis_off()
    
fig.tight_layout()
plt.show()


In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 6))


plt.scatter(merged_gdf['Index of Multiple Deprivation (IMD) Rank_z'], merged_gdf['IMD_Rank_lag_z'], color='slateblue', alpha=0.7)


plt.axhline(0, color='grey', linestyle='--')
plt.axvline(0, color='grey', linestyle='--')


ax.set_title("Moran Scatterplot of IMD Rank in London (2019)", fontsize=13)
ax.set_xlabel("Standardised IMD Rank (z-score)", fontsize=11)
ax.set_ylabel("Spatial Lag of IMD Rank (z-score)", fontsize=11)


plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 6))


plt.scatter(
    merged_gdf['Index of Multiple Deprivation (IMD) Rank_z'], 
    merged_gdf['IMD_Rank_lag_z'], 
    alpha=0.6, 
    color='slateblue'
)


ax.axvline(0, color='k', linestyle='--', alpha=0.6)
ax.axhline(0, color='k', linestyle='--', alpha=0.6)


plt.text(1, 1.5, "HH", fontsize=20, color='darkred')
plt.text(1, -1.5, "HL", fontsize=20, color='darkorange')
plt.text(-1.75, 1.5, "LH", fontsize=20, color='darkgreen')
plt.text(-1.75, -1.5, "LL", fontsize=20, color='navy')


ax.set_title("Moran Scatterplot of IMD Deciles in London (2019)", fontsize=13)
ax.set_xlabel("Standardised IMD Rank (z-score)", fontsize=11)
ax.set_ylabel("Spatial Lag of IMD Rank (z-score)", fontsize=11)

plt.tight_layout()
plt.show()

In [ ]:
def assign_quadrant(row):
    if row['Index of Multiple Deprivation (IMD) Rank_z'] > 0:
        if row['IMD_Rank_lag_z'] > 0:
            return 'HH'  
        else:
            return 'HL'  
    else:
        if row['IMD_Rank_lag_z'] > 0:
            return 'LH'  
        else:
            return 'LL'  

merged_gdf['quadrant'] = merged_gdf.apply(assign_quadrant, axis=1)

In [ ]:
qcolors = {'HH':'red', 'HL':'pink', 'LH':'lightblue', 'LL':'blue'}
cmap = colors.ListedColormap(qcolors.values())

fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# --- Choropleth Map of Moran Quadrants ---
merged_gdf.plot(
    column='quadrant',
    categorical=True,
    cmap=cmap,
    edgecolor='white',
    linewidth=0.2,
    legend=True,
    legend_kwds={"title": "Moran I Quadrant", "loc": 3},
    ax=ax[0]
)
ax[0].set_title("Local Spatial Clusters (IMD Rank)")
ax[0].axis("off")

# --- Moran Scatterplot ---
ax[1].scatter(
    merged_gdf['Index of Multiple Deprivation (IMD) Rank_z'],
    merged_gdf['IMD_Rank_lag_z'],
    c=merged_gdf['quadrant'].map(qcolors),
    alpha=0.7
)
ax[1].axvline(0, color='k', linestyle='--', alpha=0.5)
ax[1].axhline(0, color='k', linestyle='--', alpha=0.5)

# Label axes and title
ax[1].set_xlabel("Crime Rate (z)")
ax[1].set_ylabel("Spatial Lag of Crime Rate (z)")
ax[1].set_title("Moran Scatterplot")

plt.tight_layout()
plt.show()

In [ ]:
lisa = esda.moran.Moran_Local(merged_gdf['Index of Multiple Deprivation (IMD) Rank'], w)

In [ ]:
merged_gdf['Is'] = lisa.Is

fig, ax = plt.subplots(1, figsize=(9, 9))
merged_gdf.plot(column='Is', cmap='viridis',
scheme='quantiles', k=5,
linewidth=0.,
legend=True, legend_kwds={"title":"Local Moran's I","loc": 3}, ax=ax
       )
ax.set_axis_off()

In [ ]:
plt.hist(lisa.Is, 10, facecolor='lightblue', edgecolor='black')
plt.xlabel("Local Moran's I")
plt.ylabel("Count")
plt.show()

In [ ]:
plt.hist(lisa.p_sim, 10, facecolor='lightblue', edgecolor='black')
plt.xlabel("Local Moran's I p-value")
plt.ylabel("Count")
plt.show()

In [ ]:
merged_gdf['p-sim'] = lisa.p_sim
sig = 1 * (lisa.p_sim < 0.05) 
slabels = ['non-sig.', 'significant'] 
labels = [slabels[i] for i in sig]
merged_gdf['sig'] = labels
merged_gdf[['sig','p-sim']].head(35)

In [ ]:
fig, ax = plt.subplots(1,figsize=(10,8))
sigcolors = {'non-sig.':'lightgrey', 'significant':'black'}
merged_gdf.plot(column='sig', categorical=True, cmap=colors.ListedColormap(sigcolors. 
    values()),
            k=2, linewidth=0.1, edgecolor='white',
               legend=True, legend_kwds={"title":'Local Moran I',"loc": 2}, ax=ax)
plt.show()

In [ ]:
merged_gdf['sigIs'] = np.where(merged_gdf['sig'] == 'significant', merged_gdf['Is'], np.nan)


fig, ax = plt.subplots(1, figsize=(10, 8))

merged_gdf.plot(color='lightgrey', linewidth=0.0, ax=ax)


merged_gdf.plot(
    column='sigIs',
    cmap='viridis',
    scheme='quantiles',
    k=5,
    edgecolor='white',
    linewidth=0.5,
    legend=True,
    legend_kwds={"title": "Local Moran's I", "loc": "lower left"},
    ax=ax
)

# Clean up display
ax.set_title("Statistically Significant Local Moran's I – Crime Rate", fontsize=14)
ax.set_axis_off()
plt.show()

In [ ]:
merged_gdf['sigIs'] = np.where(merged_gdf['sig'] == 'significant', merged_gdf['Is'], np.nan)

# Plotting
fig, ax = plt.subplots(1, figsize=(10, 8))

# Base map (grey background)
merged_gdf.plot(color='lightgrey', linewidth=0.2, ax=ax)

# Overlay: only significant LISA values
merged_gdf.plot(
    column='sigIs',
    cmap='viridis',
    scheme='quantiles',
    k=5,
    edgecolor='white',
    linewidth=0.2,
    legend=True,
    legend_kwds={"title": "Local Moran's I", "loc": 2},
    ax=ax
)

ax.set_title("Significant Local Moran's I for Crime Rate (2019)", fontsize=14)
ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
qcolors = {'HH': 'red', 'HL': 'pink', 'LL': 'lightblue', 'LH': 'blue'}
cmap_quadrant = colors.ListedColormap(qcolors.values())


fig, ax = plt.subplots(1, 2, figsize=(12, 6))


merged_gdf.plot(
    column='quadrant',
    categorical=True,
    cmap=cmap_quadrant,
    edgecolor='white',
    linewidth=0.2,
    legend=True,
    legend_kwds={"title": 'Moran I Quadrant', "loc": 3},
    ax=ax[0]
)
ax[0].set_title("Moran's I Clusters (IMD Rank)")
ax[0].axis('off')


merged_gdf.plot(
    color='lightgrey',
    linewidth=0.2,
    ax=ax[1]
)
merged_gdf.plot(
    column='sigIs',
    cmap='viridis',
    scheme='quantiles',
    k=5,
    edgecolor='white',
    linewidth=0.2,
    legend=True,
    legend_kwds={"title": "Local Moran's I", "loc": 3},
    ax=ax[1]
)
ax[1].set_title("Significant Local Moran's I (IMD Rank)")
ax[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
q_labels = ['HH', 'LH', 'LL', 'HL']  


labels = [q_labels[i - 1] for i in lisa.q]


merged_gdf['quadrant'] = labels

sig = lisa.p_sim < 0.05


hotspot   = 1 * (sig * (lisa.q == 1))  
doughnut  = 2 * (sig * (lisa.q == 2))  
coldspot  = 3 * (sig * (lisa.q == 3))  
diamond   = 4 * (sig * (lisa.q == 4))  


spots = hotspot + doughnut + coldspot + diamond


spot_labels = ['0 non-sig.', '1 hot spot', '2 doughnut', '3 cold spot', '4 diamond']
labels = [spot_labels[i] for i in spots]


merged_gdf['slabels'] = labels

fig, ax = plt.subplots(1, figsize=(12, 12))


sig_colors = colors.ListedColormap(['lightgrey', 'red', 'lightblue', 'blue', 'pink'])


merged_gdf.plot(
    column='slabels',
    categorical=True,
    cmap=sig_colors,
    k=2,
    linewidth=0.1,
    edgecolor='white',
    legend=True,
    legend_kwds={'title': "LISA (Moran's I)", 'loc': 2},
    ax=ax
)

ax.set_axis_off()
ax.set_title("LISA Cluster Map (IMD Ranking)", fontsize=15)
plt.show()

In [ ]:
df = pd.read_csv('ID 2019 for London.csv')
print(df.head())

In [ ]:
df.rename(columns={
    'Index of Multiple Deprivation (IMD) Score': 'IMD_Score'
}, inplace=True)

In [ ]:
imd_data = df[['LSOA code (2011)', 'IMD_Score']].copy()

In [ ]:
import geopandas as gpd


gdf = gpd.read_file('LSOA_2011_London_gen_MHW.shp')  


gdf = gdf.merge(imd_data, left_on='LSOA11CD', right_on='LSOA code (2011)')

In [ ]:
w = weights.KNN.from_dataframe(merged_gdf, k=8)
w.transform = 'R'

In [ ]:
from esda.moran import Moran
import matplotlib.pyplot as plt
moran = Moran(gdf['IMD_Score'], w)

In [ ]:
round(moran.I,3)

In [ ]:
moran.p_sim

In [ ]:
plt.hist(moran.sim, 10, facecolor='lightblue', edgecolor='black')
plt.vlines(moran.I, 0, 350, color='r', linestyle="--")
plt.vlines(moran.EI, 0, 350)
plt.xlabel("Moran's I")
plt.ylabel("Count")
plt.show()

In [ ]:
from libpysal.weights import spatial_lag
gdf['IMD_Score_lag'] = spatial_lag.lag_spatial(w, gdf['IMD_Score'])

In [ ]:
def standardize(df, var):
    newname = var + '_z'
    df[newname] = (df[var] - df[var].mean()) / df[var].std()

standardize(gdf,'IMD_Score')
standardize(gdf, 'IMD_Score_lag')

In [ ]:
gdf.tail()

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(8,10))

ax=ax.flatten()


for i, p in enumerate(['IMD_Score',
                       'IMD_Score_z','IMD_Score_lag','IMD_Score_lag_z']):
    
    gdf.plot(column=p, cmap='viridis', 
                   scheme='quantiles', k=5,
                   linewidth=0.,
                   legend=True, legend_kwds={"title":p, "loc": 3}, ax=ax[i]
                  )
    ax[i].set_axis_off()
    
fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 6))


plt.scatter(gdf['IMD_Score_z'], gdf['IMD_Score_lag_z'], color='slateblue', alpha=0.7)


plt.axhline(0, color='grey', linestyle='--')
plt.axvline(0, color='grey', linestyle='--')


ax.set_title("Moran Scatterplot of IMD Score in London (2019)", fontsize=13)
ax.set_xlabel("Standardised IMD Score (z-score)", fontsize=11)
ax.set_ylabel("Spatial Lag of IMD Score (z-score)", fontsize=11)


plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 6))


plt.scatter(
    gdf['IMD_Score_z'], 
    gdf['IMD_Score_lag_z'], 
    alpha=0.6, 
    color='slateblue'
)


ax.axvline(0, color='k', linestyle='--', alpha=0.6)
ax.axhline(0, color='k', linestyle='--', alpha=0.6)


plt.text(1, 1.5, "HH", fontsize=20, color='darkred')
plt.text(1, -1.5, "HL", fontsize=20, color='darkorange')
plt.text(-1.75, 1.5, "LH", fontsize=20, color='darkgreen')
plt.text(-1.75, -1.5, "LL", fontsize=20, color='navy')


ax.set_title("Moran Scatterplot of IMD Score London LSOAs (2019)", fontsize=13)
ax.set_xlabel("Standardised IMD Score (z-score)", fontsize=11)
ax.set_ylabel("Spatial Lag of IMD Score (z-score)", fontsize=11)

plt.tight_layout()
plt.show()

In [ ]:
def assign_quadrant(row):
    if row['IMD_Score_z'] > 0:
        if row['IMD_Score_lag_z'] > 0:
            return 'HH'  
        else:
            return 'HL'  
    else:
        if row['IMD_Score_lag_z'] > 0:
            return 'LH'  
        else:
            return 'LL' 


gdf['quadrant'] = gdf.apply(assign_quadrant, axis=1)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))


gdf.plot(
    column='quadrant',
    cmap=cmap,
    edgecolor='white',
    linewidth=0.2,
    legend=True,
    legend_kwds={"title": "Moran I Quadrant", "loc": 3},
    ax=ax[0]
)
ax[0].set_title("Local Spatial Clusters (IMD Score)")
ax[0].axis("off")


ax[1].scatter(
    gdf['IMD_Score_z'],
    gdf['IMD_Score_lag_z'],
    c=gdf['quadrant'].map(qcolors),
    alpha=0.7
)
ax[1].axvline(0, color='k', linestyle='--', alpha=0.5)
ax[1].axhline(0, color='k', linestyle='--', alpha=0.5)

# Label axes and title
ax[1].set_xlabel("IMD Score (z)")
ax[1].set_ylabel("Spatial Lag of IMD Score (z)")
ax[1].set_title("Moran Scatterplot")

plt.tight_layout()
plt.show()

In [ ]:
lisa = esda.moran.Moran_Local(gdf['IMD_Score'], w)

In [ ]:
gdf['Is'] = lisa.Is

fig, ax = plt.subplots(1, figsize=(9, 9))
gdf.plot(column='Is', cmap='viridis',
scheme='quantiles', k=5,
linewidth=0.,
legend=True, legend_kwds={"title":"Local Moran's I","loc": 3}, ax=ax
       )
ax.set_axis_off()

In [ ]:
plt.hist(lisa.Is, 10, facecolor='lightblue', edgecolor='black')
plt.xlabel("Local Moran's I")
plt.ylabel("Count")
plt.show()

In [ ]:
plt.hist(lisa.p_sim, 10, facecolor='lightblue', edgecolor='black')
plt.xlabel("Local Moran's I p-value")
plt.ylabel("Count")
plt.show()

In [ ]:
gdf['p-sim'] = lisa.p_sim
sig = 1 * (lisa.p_sim < 0.05) 
slabels = ['non-sig.', 'significant'] 
labels = [slabels[i] for i in sig]
gdf['sig'] = labels
gdf[['sig','p-sim']].head(35)

In [ ]:
fig, ax = plt.subplots(1,figsize=(10,8))
sigcolors = {'non-sig.':'lightgrey', 'significant':'black'}
gdf.plot(column='sig', categorical=True, cmap=colors.ListedColormap(sigcolors. 
    values()),
            k=2, linewidth=0.1, edgecolor='white',
               legend=True, legend_kwds={"title":'Local Moran I',"loc": 2}, ax=ax)
plt.show()

In [ ]:
gdf['sigIs'] = np.where(gdf['sig'] == 'significant', gdf['Is'], np.nan)


fig, ax = plt.subplots(1, figsize=(10, 8))


gdf.plot(color='lightgrey', linewidth=0.0, ax=ax)


gdf.plot(
    column='sigIs',
    cmap='viridis',
    scheme='quantiles',
    k=5,
    edgecolor='white',
    linewidth=0.5,
    legend=True,
    legend_kwds={"title": "Local Moran's I", "loc": "lower left"},
    ax=ax
)

# Clean up display
ax.set_title("Statistically Significant Local Moran's I – IMD Score", fontsize=14)
ax.set_axis_off()
plt.show()

In [ ]:
gdf['sigIs'] = np.where(gdf['sig'] == 'significant', gdf['Is'], np.nan)


fig, ax = plt.subplots(1, figsize=(10, 8))


gdf.plot(color='lightgrey', linewidth=0.2, ax=ax)


gdf.plot(
    column='sigIs',
    cmap='viridis',
    scheme='quantiles',
    k=5,
    edgecolor='white',
    linewidth=0.2,
    legend=True,
    legend_kwds={"title": "Local Moran's I", "loc": 2},
    ax=ax
)

ax.set_title("Significant Local Moran's I for IMD Score (2019)", fontsize=14)
ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
qcolors = {'HH': 'red', 'HL': 'pink', 'LL': 'lightblue', 'LH': 'blue'}
cmap_quadrant = colors.ListedColormap(qcolors.values())


fig, ax = plt.subplots(1, 2, figsize=(12, 6))


gdf.plot(
    column='quadrant',
    categorical=True,
    cmap=cmap_quadrant,
    edgecolor='white',
    linewidth=0.2,
    legend=True,
    legend_kwds={"title": 'Moran I Quadrant', "loc": 3},
    ax=ax[0]
)
ax[0].set_title("Moran's I Clusters (IMD Score)")
ax[0].axis('off')

# --- Right: Significant Local Moran's I Values ---
gdf.plot(
    color='lightgrey',
    linewidth=0.2,
    ax=ax[1]
)
gdf.plot(
    column='sigIs',
    cmap='viridis',
    scheme='quantiles',
    k=5,
    edgecolor='white',
    linewidth=0.2,
    legend=True,
    legend_kwds={"title": "Local Moran's I", "loc": 3},
    ax=ax[1]
)
ax[1].set_title("Significant Local Moran's I (IMD Score)")
ax[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
q_labels = ['HH', 'LH', 'LL', 'HL']  

labels = [q_labels[i - 1] for i in lisa.q]


gdf['quadrant'] = labels

sig = lisa.p_sim < 0.05


hotspot   = 1 * (sig * (lisa.q == 1))  
doughnut  = 2 * (sig * (lisa.q == 2))  
coldspot  = 3 * (sig * (lisa.q == 3))  
diamond   = 4 * (sig * (lisa.q == 4)) 


spots = hotspot + doughnut + coldspot + diamond


spot_labels = ['0 non-sig.', '1 hot spot', '2 doughnut', '3 cold spot', '4 diamond']
labels = [spot_labels[i] for i in spots]


gdf['slabels'] = labels

fig, ax = plt.subplots(1, figsize=(12, 12))


sig_colors = colors.ListedColormap(['lightgrey', 'red', 'lightblue', 'blue', 'pink'])


gdf.plot(
    column='slabels',
    categorical=True,
    cmap=sig_colors,
    k=2,
    linewidth=0.1,
    edgecolor='white',
    legend=True,
    legend_kwds={'title': "LISA (Moran's I)", 'loc': 2},
    ax=ax
)

ax.set_axis_off()
ax.set_title("LISA Cluster Map (IMD Score, London LSOAs (2019))", fontsize=15)
plt.show()